# Workspace 

In [1]:
from main import OpenBookQADataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = OpenBookQADataset("train_complete.jsonl", tokenizer)

print("Loaded", len(dataset), "training examples.")
sample_encs, label = dataset[0]
print("Example input_ids shape:", sample_encs[0]["input_ids"].shape)
print("Label:", label)


Loaded 4957 training examples.
Example input_ids shape: torch.Size([1, 128])
Label: 3


In [2]:
from main import OpenBookQADataset, CustomBertMC
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = OpenBookQADataset("train_complete.jsonl", tokenizer)

batch = [dataset[i] for i in range(4)]
encodings_batch = [item[0] for item in batch]

model = CustomBertMC().to("cpu")
with torch.no_grad():
    logits = model(encodings_batch)

print("Logits shape:", logits.shape)  # should be [4, 4]
print("Logits:", logits)


Logits shape: torch.Size([4, 4])
Logits: tensor([[0.4618, 0.4196, 0.2734, 0.3441],
        [0.2386, 0.2710, 0.1938, 0.1903],
        [0.4033, 0.3238, 0.4280, 0.3636],
        [0.1562, 0.3797, 0.2178, 0.2392]])


# Task 1

In [1]:
## NOTE vanilla tokenizer testing loader

from main import OpenBookQADataset, evaluate, custom_collate
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
test_set = OpenBookQADataset("test_complete.jsonl", tokenizer)
test_loader = DataLoader(test_set, batch_size=4, collate_fn=custom_collate)

In [2]:
from main import train

trained_model = train()

test_acc, test_infer_time = evaluate(trained_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f} | Test Inference Time: {test_infer_time:.2f}s")


Starting training...
Epoch 1 | Loss: 1248.72
Epoch 2 | Loss: 649.49
Epoch 3 | Loss: 230.61
Validation Accuracy: 0.652 | Inference Time: 7.25s
Training Time: 770.04s
Test Accuracy: 0.650 | Test Inference Time: 7.37s


# Task 2:

In [2]:
from adapter_finetune import train_adapters

trained_adapter_model = train_adapters()

test_acc, test_infer_time = evaluate(trained_adapter_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f} | Test Inference Time: {test_infer_time:.2f}s")


Number of learnable parameters: 2,389,249
Starting Adapter Tuning...


KeyboardInterrupt: 

# Task 3:

In [1]:
# import torch
# with torch.no_grad():
#     torch.cuda.empty_cache()

from lora_finetune import train_lora

trained_lora_model = train_lora()

test_acc, test_infer_time = evaluate(trained_lora_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f} | Test Inference Time: {test_infer_time:.2f}s")


Number of learnable parameters: 1772548
Starting LoRA Tuning...
Epoch 1 | Loss: 3443.01


KeyboardInterrupt: 

# Task 4:

In [ ]:
## NOTE prefix test loader

from main import OpenBookQADataset, evaluate, custom_collate
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
test_set = OpenBookQADataset("test_complete.jsonl", tokenizer)
test_loader = DataLoader(test_set, batch_size=4, collate_fn=custom_collate)

In [ ]:
from prefix_finetune import train_prefix

trained_prefix_model = train_prefix()

test_acc, test_infer_time = evaluate(trained_prefix_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f} | Test Inference Time: {test_infer_time:.2f}s")


Starting Prefix Tuning...


TypeError: list indices must be integers or slices, not tuple

In [3]:
import torch.nn as nn

B = nn.Linear(2,2, bias=False)
# zero weights of B
nn.init.zeros_(B.weight)
print(B.weight)

Parameter containing:
tensor([[0., 0.],
        [0., 0.]], requires_grad=True)
